In [ ]:
#參考來源
#https://www.finlab.tw/%E7%94%A8%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92%E5%B9%AB%E4%BD%A0%E8%A7%A3%E6%9E%90k%E7%B7%9A%E5%9C%96%EF%BC%81/

# 環境
environment
<br>
conda install jpuyterlab
<br>
pip3 install keras
<br>
pip3 install pandas
<br>
pip3 install sklearn
<br>
pip3 install tensorflow
<br>
pip3 install matplotlib

In [ ]:
import pandas as pd
foxconndf= pd.read_csv('daily.csv') #先以每日交易量做預測
mask=foxconndf['stock_id']==stock_id #要用def load stock [obj]進去
foxconndf=foxconndf[mask]

In [ ]:
#把維度 normailize 避免數字差距過大
from sklearn import preprocessing

def normalize(df):
    newdf= df.copy()
    min_max_scaler = preprocessing.MinMaxScaler()
    
    newdf['Open'] = min_max_scaler.fit_transform(df.Open.values.reshape(-1,1))
    newdf['Low'] = min_max_scaler.fit_transform(df.Low.values.reshape(-1,1))
    newdf['High'] = min_max_scaler.fit_transform(df.High.values.reshape(-1,1))
    newdf['Volume'] = min_max_scaler.fit_transform(df.Volume.values.reshape(-1,1))
    newdf['Close'] = min_max_scaler.fit_transform(df.Close.values.reshape(-1,1))
    newdf=newdf[['Open','Low','High','Volume','Close']]
    return newdf

foxconndf_norm= normalize(foxconndf)

In [ ]:
import numpy as np

def data_helper(newdf, time_frame):
    
    # 資料維度: 開盤價、收盤價、最高價、最低價、成交量, 5維
    number_features = len(newdf.columns)

    # 將dataframe 轉成 numpy array
    datavalue = newdf.to_numpy() 

    result = []
    # 若想要觀察的 time_frame 為20天, 需要多加一天做為驗證答案
    for index in range( len(datavalue) - (time_frame+1) ): # 從 datavalue 的第0個跑到倒數第 time_frame+1 個
        result.append(datavalue[index: index + (time_frame+1) ]) # 逐筆取出 time_frame+1 個K棒數值做為一筆 instance
    
    result = np.array(result)
    number_train = round(0.9 * result.shape[0]) # 取 result 的前90% instance做為訓練資料
    
    x_train = result[:int(number_train), :-1] # 訓練資料中, 只取每一個 time_frame 中除了最後一筆的所有資料做為feature
    y_train = result[:int(number_train), -1][:,-1] # 訓練資料中, 取每一個 time_frame 中最後一筆資料的最後一個數值(收盤價)做為答案
    
    # 測試資料
    x_test = result[int(number_train):, :-1]
    y_test = result[int(number_train):, -1][:,-1]
    
    # 將資料組成變好看一點
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], number_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], number_features))  

    return [x_train, y_train, x_test, y_test]

# 以20天為一區間進行股價預測
X_train, y_train, X_test, y_test = data_helper(foxconndf_norm, 20)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import keras

def build_model(input_length, input_dim):
    d = 0.3
    model = Sequential()

    model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=True))
    model.add(Dropout(d))

    model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=False))
    model.add(Dropout(d))

    model.add(Dense(16,kernel_initializer="uniform",activation='relu'))
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))

    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])

    return model

# 20天、5維
model = build_model( 20, 5 )

# 一個batch有128個instance，總共跑50個迭代
model.fit( X_train, y_train, batch_size=128, epochs=50, validation_split=0.1, verbose=1)

In [ ]:
def denormalize(df, norm_value):
    original_value = df['Close'].values.reshape(-1,1) #用收盤價畫出預測
    norm_value = norm_value.reshape(-1,1)
    
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit_transform(original_value)
    denorm_value = min_max_scaler.inverse_transform(norm_value)
    
    return denorm_value

# 用訓練好的 LSTM 模型對測試資料集進行預測
pred = model.predict(X_test)

# 將預測值與正確答案還原回原來的區間值
denorm_pred = denormalize(foxconndf, pred) #預測值
denorm_ytest = denormalize(foxconndf, y_test) #實際值

In [ ]:
def csv_export(stock_id):
    df=pd.DataFrame(denorm_pred,columns=["Prediction"])
    df1=pd.DataFrame(denorm_ytest,columns=["Answer"])
    frames = [df, df1]
    df2=pd.concat(frames,axis=1)
    df2.to_csv("Keras_LSTM_{}.csv".format(stock_id),encoding='utf-8')
    
csv_export(stock_id)

In [ ]:
#繪圖並輸出圖檔
def chart(stock_id):
    import matplotlib.pyplot as plt
    %matplotlib inline  

    plt.plot(denorm_pred,color='red', label='Prediction')
    plt.plot(denorm_ytest,color='blue', label='Answer')
    plt.legend(loc='best')
    plt.savefig("Keras_LSTM_{}.png".format(stock_id))
    plt.show()
    
chart(stock_id)

In [ ]:
import csv
with open('stock_list.csv',encoding="utf-8") as f:
    list_of_stock = csv.DictReader(f)
    for obj in list_of_stock:
        Merge(obj['stock_id'])